In [1]:
from keras.preprocessing.image import ImageDataGenerator, Iterator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#parameters
classes = 12
image_size = 1500
batch_size= 6
train_total = 3820
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)

In [4]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [5]:
model = Model(inputs=model_mobilenet.input, outputs=x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500, 1500, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 750, 750, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 750, 750, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 750, 750, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 750, 750, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 750, 750, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 750, 750, 16)      0         
__________

In [6]:
model.load_weights('mobilenet_from_scratch_image_1500_local_split.hdf5')

In [7]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])

In [8]:
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 930 images belonging to 12 classes.


In [9]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.087134061432203674, 0.97419354607982023]

In [9]:
# predicting test labels for pseudo- labelling

In [10]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)

Found 794 images belonging to 1 classes.


In [11]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {validation_generator.class_indices[x]: x for x in validation_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]

InternalError: Dst tensor is not initialized.
	 [[Node: _arg_input_1_0_1/_851 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_1832__arg_input_1_0_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: dense_2/Softmax/_853 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1833_dense_2/Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from random import shuffle
def test_gen(test_files, test_preds, batch_size, target_size, imggen):
    test_data = list(zip(test_files, test_preds))
    while(True):
        shuffle(test_data)
        for start in range(0, len(test_data), batch_size):
            end = min(start + batch_size, len(test_data))
            y_batch = [test_data[i][1] for i in range(start,end)]
            
            x_batch = [test_data[i][0] for i in range(start,end)]
            x_batch = [img_to_array(load_img(x, target_size=(target_size,target_size)), 'channels_last') for x in x_batch]
            x_batch = [x/255. for x in x_batch]
            x_batch = [imggen.random_transform(x) for x in x_batch]
            
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            
            yield (x_batch, y_batch)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)

In [ ]:
test_aug_generator = test_gen(['test/'+x for x in test_generator.filenames],
                              to_categorical(np.argmax(predictions, 1)),
                              2, 1500, train_datagen)

In [ ]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(1500,1500),
                    batch_size=4,
                    class_mode='categorical',
                    shuffle=True)

In [ ]:
def pseudo_lab_gen(train_gen, test_gen):
    while(True):
        train_batch = next(train_gen)
        test_batch  = next(test_gen)
        
        x_batch = np.concatenate((train_batch[0], test_batch[0]), axis=0)
        y_batch = np.concatenate((train_batch[1], test_batch[1]), axis=0)
        
        yield (x_batch, y_batch)

In [ ]:
plg = pseudo_lab_gen(train_generator, test_aug_generator)

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [12]:
callbacks = [ModelCheckpoint(filepath='mobilenet_image_1500_local_split_pseudo_label.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1),
            EarlyStopping(monitor='val_loss', patience=20, verbose=1),
            CSVLogger('./13-metrics_1.csv')]

In [ ]:
history = model.fit_generator(
                    plg,
                    steps_per_epoch = int(np.ceil(train_total/4)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100


In [10]:
model.load_weights('mobilenet_image_1500_local_split_pseudo_label.hdf5')

In [11]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.098842934744731784, 0.97526881502520657]

In [33]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)

Found 794 images belonging to 1 classes.


In [34]:
predictions_2 = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {validation_generator.class_indices[x]: x for x in validation_generator.class_indices}
predicted_classes_2 = [class_ids[x] for x in np.argmax(predictions_2, axis=1)]

In [38]:
m = [i for i,j in zip(predicted_classes,predicted_classes_2) if(i!=j)]
m

['Loose Silky-bent', 'Black-grass', 'Loose Silky-bent', 'Loose Silky-bent']

In [39]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes_2})
submission.to_csv('submission_mobilenet_size_increase_1500_pseudo_label.csv', encoding="utf8", index=False)

In [40]:
from IPython.display import FileLink
FileLink('submission_mobilenet_size_increase_1500_pseudo_label.csv')

/home/sainath/plant_seeding/submission_mobilenet_size_increase_1500_pseudo_label.csv

In [41]:
# 97.607